# Purpose
* Test models from all of the folds on the hidden dataset;
* select the best one;
* check if ensembling improves metrics.

In [1]:
import sys
sys.path.append("..")

In [2]:
from utils.support import pkl_load
from train.run import MyDataset
from train.versions.augmentation import get_augmentation
from train.versions.model import get_model

from pathlib import Path
from torch.utils.data import DataLoader
import torch
import numpy as np
from sklearn.metrics import log_loss
import pandas as pd

In [3]:
cfg = {
    
    "model_version": 4,
    "model_weights": "../output/models/hypothesis-4/fold-{fold}/model.pt",
    
    "augmentation_version": 1,
    
    "device": "cuda",
    "batch_size": 4000,
    "n_jobs": 4,
}

In [4]:
df = pkl_load(Path("../output/crossval_split/crossval_dct.pkl"))["hidden"]
dataset = MyDataset(df, get_augmentation(cfg["augmentation_version"])["valid"])
dataloader = DataLoader(dataset, batch_size=cfg["batch_size"], shuffle=False, num_workers=cfg["n_jobs"])

df.head(3)

,cew_img,original_img,label
2,../data/dataset_B_Eye_Images/closedRightEyes/c...,None,0
3,../data/dataset_B_Eye_Images/closedRightEyes/c...,None,0
8,../data/dataset_B_Eye_Images/closedLeftEyes/cl...,None,0


## Collect Predictions

In [5]:
folds = [1, 2, 3, 4, 5]
ensemble_logits = torch.zeros(len(df))
for fold in folds:
    model = get_model(cfg["model_version"], cfg["model_weights"].format(fold=fold))
    model.eval().to(cfg["device"])
    
    with torch.no_grad():    
        pred = np.array([])
        for x, y in dataloader:
            x, y = x.to(cfg["device"]), y.to(cfg["device"])
            p = model.forward(x)
            
            ensemble_logits += p.to("cpu") / len(folds)
            
            p = torch.sigmoid(p)
            p = p.to("cpu").numpy()
            pred = np.append(pred, p)

        df[f"fold_{fold}"] = pred

df["ensemble"] = torch.sigmoid(ensemble_logits).to("cpu").numpy()
df.head(3)

,cew_img,original_img,label,fold_1,fold_2,fold_3,fold_4,fold_5,ensemble
2,../data/dataset_B_Eye_Images/closedRightEyes/c...,None,0,0.058650,0.090426,0.106545,5.762548e-02,0.115808,0.082597
3,../data/dataset_B_Eye_Images/closedRightEyes/c...,None,0,0.030063,0.105267,0.064315,5.892620e-02,0.163910,0.073215
8,../data/dataset_B_Eye_Images/closedLeftEyes/cl...,None,0,0.000013,0.000027,0.000040,9.897332e-07,0.000066,0.000016


## Estimate LogLoss

In [6]:
loss = {}
for fold in folds:
    loss[fold] = log_loss(df["label"], df[f"fold_{fold}"])
loss = pd.Series(loss)
print(f"Best Fold-{loss.idxmin()}: {loss.min()}")
print(f"Ensemble: {log_loss(df.label, df.ensemble)}")

Best Fold-1: 0.07322567712208027
Ensemble: 0.0690990011655575
